#### Byte pair encoding (BPE)

In [1]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")
text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of some"
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617]


In [2]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of some


#### BPE for Unknown word

In [3]:
text = "Hi, I'm Rui Wu. <|endoftext|> nihaowoshiwurui(chinese)."
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[17250, 11, 314, 1101, 371, 9019, 18027, 13, 220, 50256, 37628, 3099, 322, 13704, 86, 333, 9019, 7, 354, 3762, 737]


In [4]:
strings = tokenizer.decode(integers)
print(strings)

Hi, I'm Rui Wu. <|endoftext|> nihaowoshiwurui(chinese).


#### Using sliding windows to sample data

In [5]:
import re
import requests
url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt"
response = requests.get(url)
raw_text = response.text
enc_text = tokenizer.encode(raw_text)
print(len(raw_text))

20479


In [6]:
enc_sample = enc_text[50:]
print("  ",enc_sample[:10])
context_size = 4 #A
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y:      {y}")

   [290, 4920, 2241, 287, 257, 4489, 64, 319, 262, 34686]
x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [7]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [8]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


#### Create a small dataset for LLM

In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(txt) #A
        for i in range(0, len(token_ids) - max_length, stride): #B
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self): #C
        return len(self.input_ids)
    def __getitem__(self, idx): #D
        return self.input_ids[idx], self.target_ids[idx]

#### Implementing an efficient data loader

In [10]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True):
    tokenizer = tiktoken.get_encoding("gpt2") #A
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride) #B
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last) #C
    return dataloader

dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader) #A
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


#### Example for batchsize > 1

In [11]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[  198,    38,   271, 10899],
        [  319,  3619,   438,   505],
        [  351,   663,  4808, 44769],
        [  804,   379,   326,  1517],
        [ 5986,    30,  2011, 20136],
        [  326,   484,   815, 25722],
        [   13,   198,   198,     1],
        [ 1781,   314,  1422,   470]])

Targets:
 tensor([[   38,   271, 10899,   550],
        [ 3619,   438,   505,   286],
        [  663,  4808, 44769,  8270],
        [  379,   326,  1517,   438],
        [   30,  2011, 20136,   373],
        [  484,   815, 25722,   683],
        [  198,   198,     1,  5779],
        [  314,  1422,   470,  1560]])


#### Text Embedding

In [12]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [13]:
input_ids = torch.tensor([2, 3, 5, 1])
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


In [16]:
output_dim = 256
vocab_size = 50257
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

max_length = 4
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=max_length)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[  284,  1234,  8737,   656],
        [  383,  8631,  3872,   373],
        [ 3347, 27846,   503,  2048],
        [  329,   502,     0,   383],
        [ 1310,  1165,   881, 40642],
        [ 2951,   319,   262,  1327],
        [  257,  2726,  6227,   284],
        [  520,  5493,  6776,   878]])

Inputs shape:
 torch.Size([8, 4])


In [17]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


#### Position Embedding

In [19]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


#### Input Embedding

In [20]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
